# Evaluate the psychosis dataset with neuropsych measures

This was my first attempt to address the whole database.  It didn't work and we worked out a few subscales.  So this notebook is probably useless at this point.

In [1]:
from __future__ import division

from sklearn.decomposition import FactorAnalysis, PCA
from sklearn.model_selection import cross_val_score
from utils.prog import log_progress as lp
import statsmodels.formula.api as smf
from numpy import transpose as T
import matplotlib.pyplot as plt
from collections import Counter
import statsmodels.api as sm
from utils import neuropsych
import scipy.stats as stats
import matplotlib as mpl
import sklearn.cluster
import seaborn as sns
import datetime as dt
import pandas as pd
import numpy as np
import palettable
import warnings
import scipy
import time
import sys
import os

%matplotlib inline
%load_ext rpy2.ipython

#cols = palettable.colorbrewer.qualitative.Dark2_8.hex_colors
cols = palettable.tableau.ColorBlind_10.hex_colors
cols += palettable.tableau.PurpleGray_6.hex_colors
cols += palettable.tableau.Tableau_10.hex_colors

#warnings.filterwarnings("ignore")
CONDIR = os.environ.get("CONDIR")

In [3]:
table = "TRAINTABLE"
RC = pd.read_csv(os.environ.get(table),low_memory=False)

## Read in training dataset

 From here on, we only work with the training dataset

In [ ]:
CONDIR = '/scratch/users/jdurnez/Psychosis_derivatives/connectivity_Joke/'
table = 'TRAINTABLE' if True else 'CLEANTABLE'

## Prepare neuropsych measures

The following functions grab and clean the data to immediately work with.  These functions are stored in utils.

In [ ]:
RC, labeltable = neuropsych.get_tables(table)
labeltable, RC_table = neuropsych.subset_tables(table=table, 
                                                RC=RC, 
                                                labeltable=labeltable)

RC_table[:5]

Now, I'm subsetting all psych variables, to include only certain scales, an only numerical values.  Then, I'm standardising the table, so that the weights of any data reduction are comparable.

## Data cleaning

Some columns pose problems for factor analysis (matrix singularity):
1. There seem to be columns that are empty for all subjects.  These should be removed.
2. There is an extremely high collinearity for certain columns (often _severity_ and _duration_ of same variable).  You can see these in the correlation plot below: the little brown squares in the ldps scale.  These should be removed or be summarised.

This can be seen in the correlation matrix of the data matrix.

In [ ]:
cormat = RC_table.corr()
cormat.fillna(0)
unique_labels = np.unique(labeltable.scale)
labels_num = [np.where(unique_labels==x)[0].tolist()[0] for val,x in enumerate(labeltable.scale)]
major_ticks = [np.min(np.where(np.array(labels_num)==x))-1 for x in range(len(np.unique(labeltable.scale)))]
minor_ticks = [np.mean(np.where(np.array(labels_num)==x))-1 for x in range(len(np.unique(labeltable.scale)))]
fig = plt.figure(figsize=(8, 6), dpi= 100, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111)
ax1 = ax.imshow(cormat,cmap = "PuOr_r",vmin=-0.2,vmax=1,aspect='auto',interpolation='nearest')
ax.set_title('correlations between neuropsych measures')
ax.set_xticks(major_ticks,minor=False)
ax.set_xticks(minor_ticks,minor=True)
ax.set_xticklabels(unique_labels,minor=True,rotation=90)
ax.set_xticklabels(unique_labels,minor=False,visible=False)
ax.set_yticks(major_ticks,minor=False)
ax.set_yticks(minor_ticks,minor=True)
ax.set_yticklabels(unique_labels,minor=True)
ax.set_yticklabels(unique_labels,minor=False,visible=False)
plt.colorbar(ax1)
plt.tight_layout()

The following funcion removed the empty columns and removes columns with really high multicollinearity (defined by the variable `cor_limit`)

In [ ]:
labeltable, FA_table = neuropsych.clean_tables(labeltable, 
                                               RC_table,
                                               cor_lim=0.97)

In [ ]:
# psydict = os.path.join(os.environ.get("TABLEDIR"),'psychosis_dict.csv')
# psydict = pd.read_csv(psydict)
# psydict['scale']=''
# for idx,row in psydict.iterrows():
#     whichscale = np.where(row.code==labeltable.code)[0]
#     psydict.set_value(idx,'scale',labeltable.scale[whichscale])

This looks better...

In [ ]:
cormat = FA_table.corr()
cormat.fillna(0)
unique_labels = np.unique(labeltable.scale)
labels_num = [np.where(unique_labels==x)[0].tolist()[0] for val,x in enumerate(labeltable.scale)]
major_ticks = [np.min(np.where(np.array(labels_num)==x))-1 for x in range(len(np.unique(labeltable.scale)))]
minor_ticks = [np.mean(np.where(np.array(labels_num)==x))-1 for x in range(len(np.unique(labeltable.scale)))]
fig = plt.figure(figsize=(8, 6), dpi= 100, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111)
ax1 = ax.imshow(cormat,cmap = "PuOr_r",vmin=-0.2,vmax=1,aspect='auto',interpolation='nearest')
ax.set_title('correlations between neuropsych measures')
# ax.set_xticks(major_ticks,minor=False)
# ax.set_xticks(minor_ticks,minor=True)
# ax.set_xticklabels(unique_labels,minor=True,rotation=90)
# ax.set_xticklabels(unique_labels,minor=False,visible=False)
# ax.set_yticks(major_ticks,minor=False)
# ax.set_yticks(minor_ticks,minor=True)
# ax.set_yticklabels(unique_labels,minor=True)
# ax.set_yticklabels(unique_labels,minor=False,visible=False)
plt.colorbar(ax1)
plt.tight_layout()

## First factor analysis pass

We'll try a factor analysis on the whole table.  **Note: $p>n$**, the number of features is larger than the number of measurements.  As such, it will be hard to fit a factor analysis on the whole matrix.  Even if it would be possible, the results are very unstable, what can be seen in the example below.  Including more than one factor leads to a high variability of the cross-validation score.

In [20]:
def factor_ncomponents(X,n_components,cvgroups=3):
    fa = FactorAnalysis()
    fa.svd_method='randomized'
    fa.iterated_power=2
    fa.random_state = 100

    fa_mean_scores = []
    fa_std_scores = []
    maxlog = []
    for n in lp(n_components):
        fa.n_components = n
        cvscore = cross_val_score(fa, X,cv=cvgroups)
        fa_mean_scores.append(np.mean(cvscore))
        fa_std_scores.append(np.std(cvscore))
        fa.fit(X)
        maxlog.append(fa.loglike_[-1])

    return fa_mean_scores, fa_std_scores, maxlog

def plot_components(fa_mean_scores, fa_std_scores, maxlog):
    fa_z = (np.array(fa_mean_scores)-np.mean(fa_mean_scores))/np.std(fa_mean_scores)
    fastd_z = np.array(fa_std_scores)/np.std(fa_mean_scores)/len(fa_mean_scores)
    maxlog_z = (np.array(maxlog)-np.mean(maxlog))/np.std(maxlog)
    n_components_fa = n_components[np.argmax(fa_mean_scores)]
    n_components_ML = n_components[np.argmax(maxlog_z)]

    plt.figure()
    plt.plot(n_components, maxlog_z, cols[0], label='training likelihood')
    plt.axvline(n_components_ML, color=cols[0],
                label='FactorAnalysis loglikelihood: %d' % n_components_ML,
                linestyle='--')
    plt.plot(n_components, fa_z, cols[1], label='testing likelihood')
    #plt.plot(n_components, fa_z-2*fastd_z, cols[1], label='95 CI',linestyle=':')
    #plt.plot(n_components, fa_z+2*fastd_z, cols[1],linestyle=':')
    plt.axvline(n_components_fa, color=cols[1],
                label='FactorAnalysis CV: %d' % n_components_fa,
                linestyle='--')

    plt.xlabel('Number of components')
    plt.ylabel('Standardized scores')
    plt.legend(loc='lower right')
    plt.title("Number of components based on cross-validation and maximum likelihood")

The measures that are shows are the loglikelihood of the training data (thus the obtained maximum loglikelihood, which can be expected to keep going up with more factors) and the loglikelihood of the testing data given the model.

In [ ]:
#n_components = np.arange(1,15,2)

ncomponents = pd.read_csv(os.path.join(os.environ.get("MLDIR"),'ncomponents.csv'))

fig = plt.figure(figsize=(8,8))
plt.plot(ncomponents.n_components[:30],ncomponents.fa_scores[:30])
plt.title("Leave-1-out cross validation loglikelihood score for number of components.")
plt.xlabel("number of components")
plt.ylabel('CV loglikelihood')
#fa_mean_scores, fa_std_scores, maxlog = factor_ncomponents(X,n_components,cvgroups=3)

The following figure shows that already after one factor, the cross-validation loglikelihood reduced.  Given the high dimensionality of this dataset, this is very increadible.

In [ ]:
fa = FactorAnalysis()
fa.svd_method='randomized'
fa.iterated_power=3
fa.random_state = 100
fa.n_components = 6
fa.fit(FA_table)

In [ ]:
X_red = fa.components_

In [ ]:
codes = list(FA_table.columns)
labels = [psych_labels[x] for x in codes]

In [ ]:
# this code makes nice figures of factors and their loadings
minor_ticks = np.arange(len(labels))

labelnew = [x[:50] for x in labels]
fig = plt.figure(figsize=(10, 30), dpi= 100, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111)
ax1 = ax.imshow(np.transpose(X_red),
                aspect='auto',interpolation='nearest',cmap='coolwarm')
ax.set_yticks(np.arange(X_red.shape[1]),minor=True)
ax.set_yticklabels(labelnew,minor=True)
plt.colorbar(ax1)


# coldict = {k:cols[idx] for idx,k in enumerate(np.unique(labeltable.scale))}
# labeltable['colors'] = [coldict[x] for x in labeltable.scale]
# for ytick, color in zip(ax.get_yticklabels(minor=True), labeltable.colors):
#     ytick.set_color(color)

plt.tight_layout()

In [ ]:
#%%R -i FA_table -o factortable
#
#maxfac <- 5
#
#library(psych)
#factortable <- nfactors(FA_table,n=maxfac,rotation='varimax')$vss.stats

## Try to fit a data reduction at the scale of the scales :-) .
We'll first try to fit a factor analysis to each scale, starting with the legal issues. 

In [ ]:
include = ['legal_issues']
labeltable, legal_table = neuropsych.subset_tables(include=include,table=table)
labeltable, legal_table = neuropsych.clean_tables(labeltable, legal_table,cor_lim=1)

The following chunk of code performs a factor analysis in R (in R not python because R gives a lot more information about the number of factors).

In [ ]:
%%R -i legal_table -o factortable

maxfac <- min(dim(legal_table)[2],50)

#library(psych)
factortable <- nfactors(legal_table,n=maxfac,rotation='varimax')$vss.stats
factortable['eigen'] <- eigen(cor(legal_table))$values[1:maxfac]

parallel <- fa.parallel(legal_table,plot=FALSE)
factortable['parallel_data'] <- parallel$fa.values[1:maxfac]
factortable['parallel_sim'] <- parallel$fa.sim[1:maxfac]

vss <- vss(legal_table,n=maxfac,rotate='varimax',plot=FALSE)
factortable['map'] <- vss$map

In [ ]:
from utils import factanal
reload(factanal)

minfact = factanal.evaluate_factors(factortable)

In [ ]:
fig = plt.figure(figsize=(10,10))
sns.distplot(minfact.values(),bins=range(25))
plt.xlim([0,25])
plt.title("number of factors for factor analysis on legal table")
plt.xlabel('n components')
plt.ylabel('density')

Based on these scores, it's hard to pinoint the number of factors to be considered.  Let's look at the cross-validation scores.

In [ ]:
X = legal_table
n_components = np.arange(0,10,2)

fa_mean_scores, fa_std_scores, maxlog = factor_ncomponents(X,n_components,cvgroups=3)

In [ ]:
plot_components(fa_mean_scores, fa_std_scores, maxlog)

minfact['loglikelihood']=n_components[np.where(max(maxlog)==maxlog)]
minfact['cross-validation']=n_components[np.where(max(fa_mean_scores)==fa_mean_scores)]

All metrics point in different directions.  The cross-validation metric shows that 1 factor should be enough, as does the minimum average partial correlation method (known to be less conservative).  So let's go for the median: 10 factors.  Probably that's too much, but that should show up later.

Objective function:

$log(trace ((FF'+U2)^{-1} R) - log(|(FF'+U2)^-1 R|) - n.items$

In [ ]:
%%R -i legal_table -o fa_loadings

library(psych)

factortable <- fa(legal_table,nfactors=2,rotate='cluster')
fa_loadings <- factortable$loadings

In [ ]:
X=np.dot(legal_table,fa_loadings)

The factor analysis seems to be summarised in the seriousness of the legal history.

In [ ]:
minor_ticks = np.arange(len(labeltable.code))

labelnew = [x[:50] for x in legal_table.columns]
fig = plt.figure(figsize=(5, 6), dpi= 100, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111)
ax1 = ax.imshow(fa_loadings,
                aspect='auto',interpolation='nearest',cmap='coolwarm')
ax.set_yticks(np.arange(fa_loadings.shape[0]),minor=True)
ax.set_yticklabels(labelnew,minor=True)
plt.colorbar(ax1)


coldict = {k:cols[0] for idx,k in enumerate(np.unique(labeltable.scale))}
labeltable['colors'] = [coldict[x] for x in labeltable.scale]
for ytick, color in zip(ax.get_yticklabels(minor=True), labeltable.colors):
    ytick.set_color(color)

plt.tight_layout()

# Family

In [ ]:
include = ['family_history_assessment']
labeltable, family_table = neuropsych.subset_tables(include=include,table=table)
labeltable, family_table = neuropsych.clean_tables(labeltable, family_table,cor_lim=0.99)

In [ ]:
%%R -i legal_table -o factortable

maxfac <- min(dim(legal_table)[2],50)

library(psych)
factortable <- nfactors(legal_table,n=maxfac,rotation='varimax')$vss.stats
factortable['eigen'] <- eigen(cor(legal_table))$values[1:maxfac]

parallel <- fa.parallel(legal_table,plot=FALSE)
factortable['parallel_data'] <- parallel$fa.values[1:maxfac]
factortable['parallel_sim'] <- parallel$fa.sim[1:maxfac]

vss <- vss(legal_table,n=maxfac,rotate='varimax',plot=FALSE)
factortable['map'] <- vss$map


In [ ]:
from utils import factanal
reload(factanal)

minfact = factanal.evaluate_factors(factortable)

In [ ]:
X = FA_table
n_components = np.arange(0,5,1)

fa_mean_scores, fa_std_scores, maxlog = factor_ncomponents(X,n_components)

In [ ]:
plot_components(fa_mean_scores, fa_std_scores, maxlog)

minfact['loglikelihood']=n_components[np.where(max(maxlog)==maxlog)]
minfact['cross-validation']=n_components[np.where(max(fa_mean_scores)==fa_mean_scores)]

In [ ]:
sns.distplot(minfact.values(),bins=range(0,25))
#np.median(minfact.values())
plt.xlim([0,25])

In [ ]:
from collections import Counter
Counter(y)

## Baseline prediction accuracy based on neuropsych variables

Right now, the SVM is having extremely high classification accuracy.  Not really that much of a surprise, given that some forms were only filled out by psych patients (verify !) and the presence of hallucinations is pretty obvious :) .

In [ ]:
RC = pd.read_csv(os.environ.get(table),low_memory=False)
y = RC.is_this_subject_a_patient


#X = np.array(FA_table)
X= FA_table
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

cv = ShuffleSplit(n_splits=50, test_size=0.2, random_state=100)
sns.set_style("white")

def plot_learning_curve(estimator,title,X,y,ylim,cv=None,n_jobs=3,train_sizes = np.linspace(0.1,1.0,10)):
    plt.figure()
    plt.title(title)
    plt.ylim(ylim)
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=4, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")
    plt.xlabel("Training sample size")
    plt.ylabel("classification accuracy")
    plt.legend()

estimator = GaussianNB()
plot_learning_curve(estimator,"Naive Bayes",X,y,ylim=[0.5,1.05],cv=cv)

estimator = LinearSVC()
plot_learning_curve(estimator,"Support Vector Classification",X,y,ylim=[0.5,1.05],cv=cv)


In [ ]:
#%%R
#leave this in: only way to install package with R --> will need :)
#install.packages("nFactors",repos='http://cran.us.r-project.org')

### Canonical correlation

In [ ]:
from sklearn.cross_decomposition import CCA
cca = CCA(n_components = 4)
cca.fit(X_new,Y)
cca.get_params()
#cca.x_scores_.shape
cca.x_weights_.shape
cca.y_weights_